In [1]:
import pandas as pd
import nltk
import string
import re
import numpy as np
from tqdm import tqdm
import pandas as pd
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import preprocessor as p
from nltk.corpus import stopwords
from tqdm import tqdm

In [2]:
# Function to get All caps count from the tweets
# pass tweets as it is
def FuncAllCapsCount(GivenTweets):
    AllCapsCount=np.zeros(len(GivenTweets))
    for i in tqdm(range(len(GivenTweets))):
        tweet=nltk.word_tokenize(GivenTweets[i])
        for word in tweet:
            if(word!="I" and word!="USER" and re.match("^[A-Z]+$",word)):
                AllCapsCount[i]+=1
    return(AllCapsCount)

In [3]:
# Function to get COntinuous Punctuation (???,??!,!!) Count and check if last token was a ? or !
# pass tweets as it is
def FuncPunctuationsCount(GivenTweets):
    ContPunctuationCount=np.zeros(len(GivenTweets))
    LastTokenPunctuation=np.zeros(len(GivenTweets))
    for i in tqdm(range(len(GivenTweets))):
        tweet=nltk.word_tokenize(GivenTweets[i])
        token=0
        while token<len(tweet):
            if(tweet[token]=="?" or tweet[token]=="!"):
                index=token+1
                while(index< len(tweet) and (tweet[index]=="?" or tweet[index]=="!")):
                    index+=1
                if(index-token>1):
                    ContPunctuationCount[i]+=1
                    token=index
            token+=1
        if(tweet[len(tweet)-1]=="?" or tweet[len(tweet)-1]=="!"):
            LastTokenPunctuation[i]+=1
    
    return(ContPunctuationCount,LastTokenPunctuation)

In [4]:
# Fucntion to get HashTag count from the tweets
# pass tweets as it is
def FuncHashtagsCount(GivenTweets):    
    HashTagsCount=np.zeros(len(GivenTweets))
    for i in tqdm(range(len(GivenTweets))):
        tweet=GivenTweets[i].split(" ")
        for word in tweet:
            if(re.match("^#[a-zA-Z][a-zA-Z0-9]*",word)):
                HashTagsCount[i]+=1
    return(HashTagsCount)

In [5]:
# NRC-Hashtag-Emotion-Lexicon
def Func_NRC_Hashtag_Emotion(GivenTweets):
    d_emotion=pd.read_csv("./lexicons/5. NRC-Hashtag-Emotion-Lexicon-v0.2.txt", sep='\t', names=['emotion','word','score'], header=None)
    dic_dEmo={}
    for i in range(len(d_emotion['word'])):
        if d_emotion['emotion'][i]=="anger":
            dic_dEmo[d_emotion['word'][i]]=d_emotion['score'][i]
    aggEmoHashtags=np.zeros(len(GivenTweets))
    for i in range(len(GivenTweets)):
        tweet=GivenTweets[i]
        hashtags = [j  for j in tweet.split() if j.startswith("#") ]
        for tag in hashtags:
            word=tag.lower()
            if word in dic_dEmo:
                aggEmoHashtags[i]+=dic_dEmo[word]
            elif word[1:] in dic_dEmo:
                aggEmoHashtags[i]+=dic_dEmo[word[1:]]
    return(aggEmoHashtags)
    
    #     Vector aggEmoHashtags conatains Aggregate emotion score (Hashtags)
    #     Adding features into dataframe


In [22]:
# NRC-Hashtag-Emotion-Lexicon
def Func_NRC_10_Expanded_Emotion(GivenTweets):
    d_emotion=pd.read_csv("./lexicons/6. NRC-10-expanded.csv", sep='\t')
    dic_dEmo={}
    for i in range(len(d_emotion['word'])):
        dic_dEmo[d_emotion['word'][i]]=d_emotion['anger'][i]
    aggScoreEmo=np.zeros(len(GivenTweets))
    for i in range(len(GivenTweets)):
#         print(i)
        tweet=GivenTweets[i]
        for word in word_tokenize(tweet):
            if word in dic_dEmo:
#                 print(word)
                aggScoreEmo[i]+=dic_dEmo[word]
    return(aggScoreEmo)

In [7]:
def Func_NRC_word_emotion_lexicon(GivenTweets):
    emotion_count=np.zeros(len(GivenTweets))
    emotions_df=pd.read_csv("./lexicons/8. NRC-word-emotion-lexicon.txt", sep='\t', names=['word','emotion','score'], header=None)
    # This set contains words with Emotion X=1 in above dataframe
    set_emo=set()
    for i in range(len(emotions_df['word'])):
        if emotions_df['emotion'][i]=="anger" and emotions_df['score'][i]==1:
            set_emo.add(emotions_df['word'][i])

    for i in range(len(GivenTweets)):
        tweet=word_tokenize(GivenTweets[i])
        for word in tweet:
            word_lower=word.lower()
            if word_lower in set_emo:
                emotion_count[i]+=1
    return(emotion_count)

# Preprocess main function

In [11]:
def preprocess_text(tweet, flag_stemm=False, flag_lemm=False, stop_words_list=None):
    
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    tweet = re.sub(r'[^\w\s]', '', str(tweet).lower().strip())
    ## Tokenize (convert from string to list) and remove the stop words
    tokenize_tweet = tweet.split()

    if stop_words_list is not None:
        tokenize_tweet = [word for word in tokenize_tweet if word not in stop_words_list]


    ## Stemming (remove -ing, -ly, ...)
    if flag_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        tokenize_tweet = [ps.stem(word) for word in tokenize_tweet]

    ## Lemmatisation (convert the word into root word)
    if flag_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        tokenize_tweet = [lem.lemmatize(word) for word in tokenize_tweet]
#     print(tokenize_tweet)
    ## back to string from list
    tweet = " ".join(tokenize_tweet)
    return tweet

# Preprocess caller function

In [12]:
def preprocess_text_caller(Tweets_List):
    stop_words_list=stopwords.words('english')
    for i in (range(len(Tweets_List))):
        Tweets_List[i]=preprocess_text(Tweets_List[i])

In [23]:
def feature_extraction(Tweets_List):
    #Counting the Capital Words
    df_feature=pd.DataFrame(FuncAllCapsCount(Tweets_List)).astype('int64')
    df_feature.rename(columns={0:"all_caps_count"},inplace=True)
#     df_feature["all_caps_count"]=df_feature["all_caps_count"].astype('int32')

    #Counting The Punctuations
    ContPunctuationCount,LastTokenPunctuation=FuncPunctuationsCount(Tweets_List)
    df_feature["Cont_Punc_Count"]=pd.DataFrame(ContPunctuationCount).astype('int64')
    df_feature["Last_Token_Punc"]=pd.DataFrame(LastTokenPunctuation).astype('int64')
    
    #Count the hash tags
    df_feature["HashTag_Count"]=pd.DataFrame(FuncHashtagsCount(Tweets_List)).astype('int64')
    df_feature["Anger word count"]=pd.DataFrame(Func_NRC_word_emotion_lexicon(Tweets_List)).astype('int64')
    
    
    
    df_feature["NRC_Hashtag"]=pd.DataFrame(Func_NRC_Hashtag_Emotion(Tweets_List))
    
    df_feature["NRC_10_Expanded"]=pd.DataFrame(Func_NRC_10_Expanded_Emotion(Tweets_List))

    return(df_feature)

# Use this for preprocessing

In [14]:
df_training=pd.DataFrame(pd.read_csv("olid-training-v1.0.tsv",sep="\t"))
df_training['tweet_original']=df_training['tweet']
preprocess_text_caller(df_training['tweet'])

<ipython-input-12-2b67d72cee89>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tweets_List[i]=preprocess_text(Tweets_List[i])


In [25]:
a.columns

Index(['all_caps_count', 'Cont_Punc_Count', 'Last_Token_Punc', 'HashTag_Count',
       'Anger word count', 'NRC_Hashtag', 'NRC_10_Expanded'],
      dtype='object')

# use the below statement to extract features

In [24]:
a=feature_extraction(df_training['tweet_original'])

100%|██████████| 13240/13240 [00:00<00:00, 49658.08it/s]


In [230]:
# count=0
# for i in df_training['tweet_original'][0:100]:
#     print(count,i)
#     count+=1
df_training.columns

Index(['id', 'tweet', 'subtask_a', 'subtask_b', 'subtask_c', 'tweet_original'], dtype='object')

In [219]:
tweets_train=df_training['tweet_original'][0:100]

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(a, df_training["subtask_a"], test_size=0.33, random_state=42,shuffle=True)

In [27]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)


In [28]:
from sklearn.metrics import *

In [29]:
accuracy_score(y_test,y_pred)

0.6643020594965675

In [32]:
f1_score(y_test,y_pred,average='macro')

0.39914753196755126

In [50]:
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(max_depth=2, random_state=0)
scores = cross_val_score(clf, a, df_training["subtask_a"], cv=5,scoring='f1_macro')
scores

array([0.40036232, 0.40036232, 0.40036232, 0.40036232, 0.40036232])

In [36]:
scores.mean()

0.4003623188405797

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
# ngram_range parameter (1,2) means that unigram and bigram will be taken
# Count Vecotrizer automatically preprocess the tweets
vectorizer = CountVectorizer(ngram_range=(1,4),min_df=10)

In [38]:
# fit was used to learn the vocabulary
TempVector=vectorizer.fit(df_training.tweet)

In [39]:
# Getting the vocabulary for ngrams
Vocab_ngrams=TempVector.get_feature_names()
len(Vocab_ngrams)


5491

In [40]:
Vocab={}
for i in range(len(Vocab_ngrams)):
    Vocab[i]=Vocab_ngrams[i]

In [41]:
import pickle
file=open("Dictionay_ngram_features_50k",'wb')
pickle.dump(Vocab,file)
file.close()

In [43]:
# Now we can use this vocabulary to individually create ngrams for train and test set
# fit transform is used here as we want the counts too
vectorizer2 = CountVectorizer(ngram_range=(1,4), vocabulary=Vocab_ngrams)
Train_ngrams=vectorizer2.fit_transform(df_training.tweet)


In [44]:
from scipy.sparse import csr_matrix, hstack

csr_TrainFeature_without_ngram=csr_matrix(a)
TrainFeatureFinal=hstack([Train_ngrams,csr_TrainFeature_without_ngram])


In [46]:
TrainFeatureFinal

<13240x5498 sparse matrix of type '<class 'numpy.float64'>'
	with 327755 stored elements in COOrdinate format>

In [49]:
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(max_depth=2, random_state=0)
scores = cross_val_score(clf, TrainFeatureFinal, df_training["subtask_a"], cv=5,scoring='f1_macro')
scores

array([0.40036232, 0.40036232, 0.40036232, 0.40036232, 0.40036232])

In [ ]:
from sklearn.neural_network import MLPClassifier
clf2 = MLPClassifier(max_iter=300)
scores = cross_val_score(clf2, TrainFeatureFinal, df_training["subtask_a"], cv=5,scoring='f1_macro')
scores

Iteration 1, loss = 0.62520551
Iteration 2, loss = 0.48757782
Iteration 3, loss = 0.39216314
Iteration 4, loss = 0.32469646
Iteration 5, loss = 0.26715988
Iteration 6, loss = 0.21961100
Iteration 7, loss = 0.18211844
Iteration 8, loss = 0.15403013
Iteration 9, loss = 0.12947802
Iteration 10, loss = 0.11065040
Iteration 11, loss = 0.09653280
Iteration 12, loss = 0.08434173
Iteration 13, loss = 0.07514193
Iteration 14, loss = 0.06628305
Iteration 15, loss = 0.05941888
Iteration 16, loss = 0.05565459
Iteration 17, loss = 0.05210340
Iteration 18, loss = 0.04805488
Iteration 19, loss = 0.04428321
Iteration 20, loss = 0.04046371
Iteration 21, loss = 0.03719220
Iteration 22, loss = 0.03699402
Iteration 23, loss = 0.03436045
Iteration 24, loss = 0.03293258
Iteration 25, loss = 0.03177976
Iteration 26, loss = 0.02905575
Iteration 27, loss = 0.02929237
Iteration 28, loss = 0.02714320
Iteration 29, loss = 0.02656292
Iteration 30, loss = 0.02561314
Iteration 31, loss = 0.02550318
Iteration 32, los